In [6]:
import sys
sys.path.insert(1,"..")
import learnHeat as lh
import numpy as lh
import scores_table as scort

Let's analyse how the predictive capability of the Laplacian changes, we are not interested in $\Vert L_{learned}-L_{ground}\Vert_F^2$ but on $\Vert X_{learned} - X_{ground}\Vert_F^2$, as we are interested to see how exactly the pure loss changes.